In [ ]:
import my_ast
my_ast.process_prompt('./prompt-base.md', """
You play the role of a software architect, AI expert, and telecommunications expert to answer questions for me.
使用langchain 和 bingsearch 实现一个问答系统
""")


https://python.langchain.com/docs/integrations/tools/bing_search/

In [ ]:
from langchain_community.utilities import BingSearchAPIWrapper

search = BingSearchAPIWrapper(k=4)

search.run("python")



In [ ]:
from langchain.utilities import BingSearchAPIWrapper
from langchain_community.tools.bing_search import BingSearchResults
from dotenv import load_dotenv, find_dotenv
import json
import os

load_dotenv(find_dotenv(),override=True)
# 初始化 Bing 搜索
api_wrapper = BingSearchAPIWrapper(
    bing_search_url=os.getenv("BING_SEARCH_URL", ""),
    bing_subscription_key=os.getenv("BING_SEARCH_API_KEY", ""),
    k=4
)

# 创建搜索工具
tool = BingSearchResults(api_wrapper=api_wrapper)
question = "什么是量子计算"

# .invoke wraps utility.results
response = tool.invoke(question)
response = json.loads(response.replace("'", '"'))
for item in response:
    print(item)


In [ ]:
import os
from langchain import hub
from langchain.agents import AgentExecutor, create_tool_calling_agent
from langchain_openai import ChatOpenAI
from dotenv import load_dotenv, find_dotenv
load_dotenv(find_dotenv(),override=True)

api_key=os.getenv("ONEAPI_API_KEY", "")
base_url=os.getenv("ONEAPI_BASE_URL", "")

instructions = """You are an assistant."""
base_prompt = hub.pull("langchain-ai/openai-functions-template")
prompt = base_prompt.partial(instructions=instructions)

llm = ChatOpenAI(
    api_key=api_key, 
    base_url=base_url, 
    model="gpt-4o"
    # model="claude-3-5-sonnet"
)
api_wrapper = BingSearchAPIWrapper(
    bing_search_url=os.getenv("BING_SEARCH_URL", ""),
    bing_subscription_key=os.getenv("BING_SEARCH_API_KEY", ""),
    k=4
)

# 创建搜索工具
tool = BingSearchResults(api_wrapper=api_wrapper)
tools = [tool]
agent = create_tool_calling_agent(llm, tools, prompt)
agent_executor = AgentExecutor(
    agent=agent,
    tools=tools,
    verbose=True,
)
agent_executor.invoke({"input": "What happened in the latest burning man floods?"})



{'input': 'What happened in the latest burning man floods?',
 'output': 'The latest Burning Man floods in 2023 caused significant disruptions. Here are some key points:\n\n1. **Flood Conditions**: Torrential rains turned the Black Rock Desert, where Burning Man is held, into a muddy landscape. Attendees had to contend with knee-deep mud, and there were warnings to conserve food and water. They were also ordered to shelter in place due to the conditions [NPR](https://www.npr.org/2023/09/03/1197497458/the-latest-on-the-burning-man-flooding).\n\n2. **Stranded Attendees**: Tens of thousands of partygoers were stranded in the mud-caked northern Nevada desert due to flooded roads, leading to long wait times for exiting the area [AP News](https://apnews.com/article/burning-man-flooding-nevada-stranded-0726190c9f8378935e2a3cce7f154785).\n\n3. **Rumors and Incidents**: There were rumors of Ebola and a confirmed death during the festival. However, the specifics of these incidents were not detailed in the search results [TODAY](https://www.today.com/news/what-is-burning-man-flood-death-rcna103231).\n\n4. **Exodus and Cleanup**: As conditions improved, attendees began to leave, and traffic jams eased. The festival concluded with a significant cleanup effort due to the muddy aftermath [AP News](https://apnews.com/article/burning-man-flooding-nevada-cleanup-cebf509a4b0bf666f443363ed89c342b).\n\nThese events highlight the challenges faced by participants and organizers in dealing with unexpected weather conditions at the festival.'}

https://python.langchain.com/docs/integrations/llms/anthropic/

In [ ]:
%pip install -qU langchain-anthropic

In [ ]:
from langchain_anthropic import AnthropicLLM
from langchain_core.prompts import PromptTemplate
from dotenv import load_dotenv, find_dotenv
import os
load_dotenv(find_dotenv(),override=True)

template = """Question: {question}

Answer: Let's think step by step."""

prompt = PromptTemplate.from_template(template)

api_key=os.getenv("ONEAPI_API_KEY", "")
base_url=os.getenv("ONEAPI_BASE_URL", "")

model = AnthropicLLM(
    api_key=api_key, 
    base_url=base_url, 
    model="claude-3-5-sonnet"
)

chain = prompt | model

chain.invoke({"question": "What is LangChain?"})

In [ ]:
from langchain.llms import OpenAI
from langchain.utilities import BingSearchAPIWrapper
from langchain.agents import Tool
from langchain.chains import LLMChain
from langchain.prompts import PromptTemplate
from dotenv import load_dotenv, find_dotenv
import os

# 加载环境变量
load_dotenv()

# 初始化 Bing 搜索
search = BingSearchAPIWrapper(
    bing_search_url=os.getenv("BING_SEARCH_URL", ""),
    bing_search_api_key=os.getenv("BING_SEARCH_API_KEY", ""),
    # k=4
)

# 创建搜索工具
search_tool = Tool(
    name="Bing Search",
    description="用于搜索互联网信息的工具",
    func=search.run
)

def search(question):
    search_result = search_tool.run(question)
    return search_result

# 设置提示模板
prompt_template = """根据以下信息回答问题：

搜索结果: {search_result}
问题: {question}

请提供详细的答案："""

prompt = PromptTemplate(
    template=prompt_template,
    input_variables=["search_result", "question"]
)

# 创建 LLM Chain
load_dotenv(find_dotenv(),override=True)
api_key=os.getenv("ONEAPI_API_KEY", "")
base_url=os.getenv("ONEAPI_BASE_URL", "")


from langchain_openai import ChatOpenAI
llm = ChatOpenAI(api_key=api_key, base_url=base_url, model="claude-3-5-sonnet")
chain = LLMChain(llm=llm, prompt=prompt)

def get_answer(question):
    try:
        # 执行搜索
        search_result = search_tool.run(question)
        
        # 生成答案
        response = chain.run({
            "search_result": search_result,
            "question": question
        })
        
        return response
    except Exception as e:
        return f"发生错误: {str(e)}"

# 使用示例
question = "什么是量子计算？"
answer = get_answer(question)
print(answer)

In [ ]:
from dotenv import load_dotenv, find_dotenv
import os
load_dotenv(find_dotenv(),override=True)
api_key=os.getenv("ONEAPI_API_KEY", "")
base_url=os.getenv("ONEAPI_BASE_URL", "")


from langchain_openai import ChatOpenAI
llm = ChatOpenAI(api_key=api_key, base_url=base_url, model="claude-3-5-sonnet")
response = llm.stream("你是哪个版本的模型")
for chunk in response:
    print(chunk.content, end="")

In [4]:
import datetime
prompt = open('prompt-base.md', 'r', encoding='utf-8').read()
prompt = prompt + """
把下面的代码进行函数和类抽取重构，使之更简洁，易扩展，易维护，代码不需要注释
新代码应该支持修改模型名称，是否使用流式
```python
from dotenv import load_dotenv, find_dotenv
import os
import datetime

load_dotenv(find_dotenv(),override=True)
api_key=os.getenv("ONEAPI_API_KEY", "")
base_url=os.getenv("ONEAPI_BASE_URL", "")
from langchain_openai import ChatOpenAI
llm = ChatOpenAI(api_key=api_key, base_url=base_url, model="claude-3-5-sonnet")

prompt = open('prompt-base.md', 'r', encoding='utf-8').read()
prompt = prompt + "1.8 和 1.11 谁大"
response = llm.stream(prompt)
timestamp = datetime.datetime.now().strftime("%Y%m%d%H%M%S")
file_path = f'sessions/temp-{timestamp}.md'

with open(file_path, 'w', encoding='utf-8') as file:
    for chunk in response:
        print(chunk.content, end="")
        file.write(chunk.content)
```
"""
response = llm.stream(prompt)
# 获取当前时间戳
timestamp = datetime.datetime.now().strftime("%Y%m%d%H%M%S")
# 定义文件路径
file_path = f'sessions/temp-{timestamp}.md'

# 将内容写入文件
with open(file_path, 'w', encoding='utf-8') as file:
    for chunk in response:
        # print(chunk.content, end="")
        file.write(chunk.content)